In [1]:
# !pip install wilds

     |████████████████████████████████| 126 kB 5.3 MB/s 
     |████████████████████████████████| 503 kB 44.7 MB/s 
     |████████████████████████████████| 38.1 MB 1.5 MB/s 
     |████████████████████████████████| 78 kB 6.2 MB/s 
     |████████████████████████████████| 4.3 MB 34.2 MB/s 
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=0f14ad6f0ce8c83a43047049f8dfca4142c36e3392727ac1563f341c22d9313e
  Stored in directory: /root/.cache/pip/wheels/d6/64/cd/32819b511a488e4993f2fab909a95330289c3f4e0f6ef4676d
Successfully built littleutils
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Attempting uninstall: pytz
    Found existing installation: pytz 2018.9
    Uninstalling pytz-2018.9:
      Successfully uninstalled pytz-2018.9
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Success

In [1]:
import numpy as np
from wilds import get_dataset
from wilds.common.data_loaders import get_train_loader
import torchvision.transforms as transforms
import torchvision.models as models
from wilds.common.data_loaders import get_eval_loader
import torch
from torch import nn
from tqdm import tqdm

In [2]:
# Load the full dataset, and download it if necessary
dataset = get_dataset(dataset="camelyon17", download=False)


In [3]:
BATCH_SIZE = 32
FRACTION = 0.33

# Get the training set
train_data = dataset.get_subset(
    "train",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(train_data)) #302436 initially
# Prepare the standard data loader
train_loader = get_train_loader("standard", train_data, batch_size=BATCH_SIZE)

"""
# (Optional) Load unlabeled data
dataset = get_dataset(dataset="camelyon17", download=True, unlabeled=True)
unlabeled_data = dataset.get_subset(
    "test_unlabeled",
    transform=transforms.Compose(
        [transforms.Resize((448, 448)), transforms.ToTensor()]
    ),
)
unlabeled_loader = get_train_loader("standard", unlabeled_data, batch_size=16)
"""
"""
# Train loop
for labeled_batch, unlabeled_batch in zip(train_loader, unlabeled_loader):
    x, y, metadata = labeled_batch
    unlabeled_x, unlabeled_metadata = unlabeled_batch
    ...
"""

99804


'\n# Train loop\nfor labeled_batch, unlabeled_batch in zip(train_loader, unlabeled_loader):\n    x, y, metadata = labeled_batch\n    unlabeled_x, unlabeled_metadata = unlabeled_batch\n    ...\n'

In [4]:
# Get the test set
id_val_data = dataset.get_subset(
    "id_val",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(id_val_data))

# Prepare the evaluation data loader
id_val_loader = get_eval_loader("standard", id_val_data, batch_size=BATCH_SIZE)


11075


In [5]:
# Get the test set
val_data = dataset.get_subset(
    "val",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(val_data))

# Prepare the evaluation data loader
val_loader = get_eval_loader("standard", val_data, batch_size=BATCH_SIZE)

11518


In [6]:
# Get the test set
test_data = dataset.get_subset(
    "test",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(test_data))

# Prepare the evaluation data loader
test_loader = get_eval_loader("standard", test_data, batch_size=BATCH_SIZE)

28068


In [7]:

# load the ResNet-18 model, with weights pretrained on ImageNet
resnet18_pretrained = models.resnet18(pretrained=True)

num_params = 0
print("Model's parameters: ")
for n, p in resnet18_pretrained.named_parameters():
    print('\t', n, ': ', p.size())
    num_params += p.numel()
print("Number of model parameters: ", num_params)

"""
formula [(W−K+2P)/S]+1.

W is the input volume
K is the Kernel size
P is the padding
S is the stride

"""

Model's parameters: 
	 conv1.weight :  torch.Size([64, 3, 7, 7])
	 bn1.weight :  torch.Size([64])
	 bn1.bias :  torch.Size([64])
	 layer1.0.conv1.weight :  torch.Size([64, 64, 3, 3])
	 layer1.0.bn1.weight :  torch.Size([64])
	 layer1.0.bn1.bias :  torch.Size([64])
	 layer1.0.conv2.weight :  torch.Size([64, 64, 3, 3])
	 layer1.0.bn2.weight :  torch.Size([64])
	 layer1.0.bn2.bias :  torch.Size([64])
	 layer1.1.conv1.weight :  torch.Size([64, 64, 3, 3])
	 layer1.1.bn1.weight :  torch.Size([64])
	 layer1.1.bn1.bias :  torch.Size([64])
	 layer1.1.conv2.weight :  torch.Size([64, 64, 3, 3])
	 layer1.1.bn2.weight :  torch.Size([64])
	 layer1.1.bn2.bias :  torch.Size([64])
	 layer2.0.conv1.weight :  torch.Size([128, 64, 3, 3])
	 layer2.0.bn1.weight :  torch.Size([128])
	 layer2.0.bn1.bias :  torch.Size([128])
	 layer2.0.conv2.weight :  torch.Size([128, 128, 3, 3])
	 layer2.0.bn2.weight :  torch.Size([128])
	 layer2.0.bn2.bias :  torch.Size([128])
	 layer2.0.downsample.0.weight :  torch.Size([12

'\nformula [(W−K+2P)/S]+1.\n\nW is the input volume\nK is the Kernel size\nP is the padding\nS is the stride\n\n'

In [8]:
# function counting the number of parameters and the number of trainable parameters of a model
# optionally, it will also display the layers
def check_model_parameters(model, display_layers=False):
  num_params = 0
  num_trainable_params = 0
  if display_layers==True:
    print("Model's parameters: ")
  for n, p in model.named_parameters():
      if display_layers == True:
        print('\t', n, ': ', p.size())
      num_params += p.numel()
      if p.requires_grad:
        num_trainable_params += p.numel()
  print("Number of model parameters: ", num_params)
  print("Number of trainable parameters: ", num_trainable_params)

In [9]:
# freeze the model parameters

# check the number of parameters and the number of trainable parameters
check_model_parameters(resnet18_pretrained, display_layers=False)

# freeze all the layers
# for param in resnet18_pretrained.parameters():
  # param.requires_grad = False

# check the number of parameters and the number of trainable parameters
check_model_parameters(resnet18_pretrained, display_layers=False)

Number of model parameters:  11689512
Number of trainable parameters:  11689512
Number of model parameters:  11689512
Number of trainable parameters:  11689512


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 5

In [11]:

resnet18_pretrained.fc = nn.Linear(in_features=512, out_features=2, bias=True)

check_model_parameters(resnet18_pretrained, display_layers=True)

Model's parameters: 
	 conv1.weight :  torch.Size([64, 3, 7, 7])
	 bn1.weight :  torch.Size([64])
	 bn1.bias :  torch.Size([64])
	 layer1.0.conv1.weight :  torch.Size([64, 64, 3, 3])
	 layer1.0.bn1.weight :  torch.Size([64])
	 layer1.0.bn1.bias :  torch.Size([64])
	 layer1.0.conv2.weight :  torch.Size([64, 64, 3, 3])
	 layer1.0.bn2.weight :  torch.Size([64])
	 layer1.0.bn2.bias :  torch.Size([64])
	 layer1.1.conv1.weight :  torch.Size([64, 64, 3, 3])
	 layer1.1.bn1.weight :  torch.Size([64])
	 layer1.1.bn1.bias :  torch.Size([64])
	 layer1.1.conv2.weight :  torch.Size([64, 64, 3, 3])
	 layer1.1.bn2.weight :  torch.Size([64])
	 layer1.1.bn2.bias :  torch.Size([64])
	 layer2.0.conv1.weight :  torch.Size([128, 64, 3, 3])
	 layer2.0.bn1.weight :  torch.Size([128])
	 layer2.0.bn1.bias :  torch.Size([128])
	 layer2.0.conv2.weight :  torch.Size([128, 128, 3, 3])
	 layer2.0.bn2.weight :  torch.Size([128])
	 layer2.0.bn2.bias :  torch.Size([128])
	 layer2.0.downsample.0.weight :  torch.Size([12

In [12]:
def train_epoch(model, train_dataloader, loss_crt, optimizer, device):
    """
    model: Model object
    train_dataloader: DataLoader over the training dataset
    loss_crt: loss function object
    optimizer: Optimizer object
    device: torch.device('cpu) or torch.device('cuda')

    The function returns:
     - the epoch training loss, which is an average over the individual batch
       losses
    """
    model.train()
    epoch_loss = 0.0
    epoch_accuracy = 0.0
    num_batches = len(train_dataloader)
    for batch_idx, batch in tqdm(enumerate(train_dataloader)):
        # shape: batch_size x 1 x 28 x 28, batch_size x 1
        # print("Train")
        # print(batch)
        batch_img, batch_labels, _ = batch
        # move data to GPU
        batch_img = batch_img.to(device)
        batch_labels = batch_labels.to(device)

        # initialize as zeros all the gradients of the model
        model.zero_grad()

        # get predictions from the FORWARD pass
        # shape: batch_size x 10
        output = model(batch_img)

        loss = loss_crt(output, batch_labels.squeeze())
        loss_scalar = loss.item()

        # BACKPROPAGATE the gradients
        loss.backward()
        # use the gradients to OPTIMISE the model
        optimizer.step()

        epoch_loss += loss_scalar

        pred = output.argmax(dim=1, keepdim=True)
        epoch_accuracy += pred.eq(batch_labels.view_as(pred)).float().mean().item()

    epoch_loss = epoch_loss/num_batches
    epoch_accuracy = 100. * epoch_accuracy/num_batches
    return epoch_loss, epoch_accuracy

def eval_epoch(model, val_dataloader, loss_crt, device):
    """
    model: Model object
    val_dataloader: DataLoader over the validation dataset
    loss_crt: loss function object
    device: torch.device('cpu) or torch.device('cuda')

    The function returns:
     - the epoch validation loss, which is an average over the individual batch
       losses
    """
    model.eval()
    epoch_loss = 0.0
    epoch_accuracy = 0.0
    num_batches = len(val_dataloader)
    with torch.no_grad():
        for batch_idx, batch in tqdm(enumerate(val_dataloader)):
            # print("Eval")
            # print(batch)
            # shape: batch_size x 3 x 28 x 28, batch_size x 1
            batch_img, batch_labels, _ = batch
            current_batch_size = batch_img.size(0)

            # move data to GPU
            batch_img = batch_img.to(device)
            batch_labels = batch_labels.to(device)

            # batch_size x 10
            output = model(batch_img)

            loss = loss_crt(output, batch_labels.squeeze())
            loss_scalar = loss.item()

            epoch_loss += loss_scalar

            pred = output.argmax(dim=1, keepdim=True)
            epoch_accuracy += pred.eq(batch_labels.view_as(pred)).float().mean().item()

    epoch_loss = epoch_loss/num_batches
    epoch_accuracy = 100. * epoch_accuracy/num_batches
    return epoch_loss, epoch_accuracy

In [13]:
resnet18_pretrained.to(device)

# create a SGD optimizer
optimizer = torch.optim.SGD(resnet18_pretrained.parameters(), lr=0.01, momentum=0.9)

# set up loss function
loss_criterion = nn.CrossEntropyLoss()

# evaluate the initial model
# val_loss, al_accuracy = eval_epoch(resnet18_pretrained, id_val_loader, loss_criterion, device)
# print('Validation performance before finetuning -- loss: %10.8f, accuracy: %10.8f'%(val_loss, val_accuracy))


In [ ]:

# finetune the model
train_losses = []
train_accuracies = []
id_val_losses = []
id_val_accuracies = []
for epoch in range(1, num_epochs+1):
  train_loss, train_accuracy = train_epoch(resnet18_pretrained, train_loader, loss_criterion, optimizer, device)
  val_loss, val_accuracy = eval_epoch(resnet18_pretrained, id_val_loader, loss_criterion, device)
  train_losses.append(train_loss)
  id_val_losses.append(val_loss)
  train_accuracies.append(train_accuracy)
  id_val_accuracies.append(val_accuracy)
  print('\nEpoch %d'%(epoch))
  print('train loss: %10.8f, accuracy: %10.8f'%(train_loss, train_accuracy))
  print('id_val loss: %10.8f, accuracy: %10.8f'%(val_loss, val_accuracy))

In [ ]:
def run_eval_ood(model, loader, loss_criterion, device, eval_type):
  losses = []
  accuracies = []

  loss, accuracy = eval_epoch(model, loader, loss_criterion, device)
  losses.append(loss)
  accuracies.append(accuracy)
  print(eval_type + ' loss: %10.8f, accuracy: %10.8f'%(loss, accuracy))

In [ ]:
run_eval_ood(resnet18_pretrained, val_loader, loss_criterion, device, "val")

360it [00:43,  8.24it/s]

val loss: 0.46796423, accuracy: 87.63715279


In [ ]:
run_eval_ood(resnet18_pretrained, test_loader, loss_criterion, device, "test")

878it [01:48,  8.10it/s]

test loss: 1.12270584, accuracy: 77.62314920


In [ ]:
densenet121_pretrained = models.densenet121(pretrained=True)

# for param in densenet121_pretrained.parameters():
  # param.requires_grad = False

check_model_parameters(densenet121_pretrained, display_layers=False)


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

Number of model parameters:  7978856
Number of trainable parameters:  7978856


In [ ]:

densenet121_pretrained.classifier = nn.Linear(in_features=1024, out_features=2, bias=True)
check_model_parameters(densenet121_pretrained, display_layers=False)

Number of model parameters:  6955906
Number of trainable parameters:  6955906


In [ ]:
densenet121_pretrained.to(device)

# create a SGD optimizer
optimizer = torch.optim.SGD(densenet121_pretrained.parameters(), lr=0.01, momentum=0.9)

# set up loss function
loss_criterion = nn.CrossEntropyLoss()

# evaluate the initial model
# val_loss, al_accuracy = eval_epoch(resnet18_pretrained, id_val_loader, loss_criterion, device)
# print('Validation performance before finetuning -- loss: %10.8f, accuracy: %10.8f'%(val_loss, val_accuracy))

# finetune the model
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
id_val_losses = []
id_val_accuracies = []

for epoch in range(1, num_epochs+1):
  train_loss, train_accuracy = train_epoch(densenet121_pretrained, train_loader, loss_criterion, optimizer, device)
  val_loss, val_accuracy = eval_epoch(densenet121_pretrained, id_val_loader, loss_criterion, device)
  train_losses.append(train_loss)
  id_val_losses.append(val_loss)
  train_accuracies.append(train_accuracy)
  id_val_accuracies.append(val_accuracy)
  print('\nEpoch %d'%(epoch))
  print('train loss: %10.8f, accuracy: %10.8f'%(train_loss, train_accuracy))
  print('id_val loss: %10.8f, accuracy: %10.8f'%(val_loss, val_accuracy))


3119it [15:59,  3.25it/s]
347it [00:48,  7.20it/s]



Epoch 1
train loss: 0.08947193, accuracy: 96.96717698
id_val loss: 0.04903783, accuracy: 98.27089337


3119it [15:52,  3.28it/s]
347it [00:47,  7.36it/s]



Epoch 2
train loss: 0.04349755, accuracy: 98.54520680
id_val loss: 0.03753684, accuracy: 98.72118156


3119it [15:50,  3.28it/s]
347it [00:47,  7.33it/s]



Epoch 3
train loss: 0.03116413, accuracy: 98.91276966
id_val loss: 0.03892054, accuracy: 98.68515850


3119it [15:50,  3.28it/s]
347it [00:46,  7.48it/s]



Epoch 4
train loss: 0.02413429, accuracy: 99.13834562
id_val loss: 0.04475565, accuracy: 98.57708934


3119it [15:48,  3.29it/s]
347it [00:47,  7.32it/s]


Epoch 5
train loss: 0.02237113, accuracy: 99.20547451
id_val loss: 0.04431844, accuracy: 98.48703170


In [ ]:
run_eval_ood(densenet121_pretrained, val_loader, loss_criterion, device, "val")

360it [00:49,  7.26it/s]

val loss: 0.61660476, accuracy: 85.78125000


In [ ]:
run_eval_ood(densenet121_pretrained, test_loader, loss_criterion, device, "test")

878it [02:00,  7.29it/s]

test loss: 0.91136280, accuracy: 79.53445330


In [ ]:
torch.save(resnet18_pretrained.state_dict(), "resnet18_pretrained.pt")

In [ ]:
torch.save(densenet121_pretrained.state_dict(), "densenet121_pretrained.pt")

In [ ]:
resnetxt50_pretrained = models.resnext50_32x4d(pretrained=True)

# freeze all the layers
# for param in resnetxt50_pretrained.parameters():
  # param.requires_grad = False

resnetxt50_pretrained.fc = nn.Linear(in_features=2048, out_features=2, bias=True)

resnetxt50_pretrained.to(device)

# create a SGD optimizer
optimizer = torch.optim.SGD(resnetxt50_pretrained.parameters(), lr=0.01, momentum=0.9)

# set up loss function
loss_criterion = nn.CrossEntropyLoss()


# finetune the model
train_losses = []
train_accuracies = []
id_val_losses = []
id_val_accuracies = []
for epoch in range(1, num_epochs+1):
  train_loss, train_accuracy = train_epoch(resnetxt50_pretrained, train_loader, loss_criterion, optimizer, device)
  val_loss, val_accuracy = eval_epoch(resnetxt50_pretrained, id_val_loader, loss_criterion, device)
  train_losses.append(train_loss)
  id_val_losses.append(val_loss)
  train_accuracies.append(train_accuracy)
  id_val_accuracies.append(val_accuracy)
  print('\nEpoch %d'%(epoch))
  print('train loss: %10.8f, accuracy: %10.8f'%(train_loss, train_accuracy))
  print('id_val loss: %10.8f, accuracy: %10.8f'%(val_loss, val_accuracy))


Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth


  0%|          | 0.00/95.8M [00:00<?, ?B/s]

3119it [17:44,  2.93it/s]
347it [00:43,  7.91it/s]



Epoch 1
train loss: 0.08606202, accuracy: 97.03115696
id_val loss: 0.05096113, accuracy: 98.17182997


3119it [17:43,  2.93it/s]
347it [00:43,  7.97it/s]


...


Epoch 3
train loss: 0.02918555, accuracy: 98.98104360
id_val loss: 0.03763097, accuracy: 98.68515850


3119it [17:42,  2.93it/s]
347it [00:44,  7.88it/s]



Epoch 4
train loss: 0.02205006, accuracy: 99.21048413
id_val loss: 0.03875422, accuracy: 98.63112392


3119it [17:44,  2.93it/s]
347it [00:43,  7.97it/s]


Epoch 5
train loss: 0.01629151, accuracy: 99.41788233
id_val loss: 0.04178602, accuracy: 98.77521614


In [ ]:
run_eval_ood(resnetxt50_pretrained, val_loader, loss_criterion, device, "val")

360it [00:45,  7.89it/s]

val loss: 0.40985703, accuracy: 87.87210650


In [ ]:
run_eval_ood(resnetxt50_pretrained, test_loader, loss_criterion, device, "test")

878it [01:51,  7.87it/s]

test loss: 1.72625949, accuracy: 72.21668565


In [ ]:
torch.save(densenet121_pretrained.state_dict(), "resnetxt50_pretrained.pt")

In [25]:
resnet18_pretrained.load_state_dict(torch.load("resnet18_pretrained_all_grad.pt"))

<All keys matched successfully>

In [53]:
"""
Rupere cap de clasificare si calculare distanta l2 si cos medie intre ood si iid
"""

# putem sa folosim labels atunci cand facem distanta? (i.e. distanta intra-clasa sau inter-clasa)

def calc_dist_iid(model, dataloader, metric="l2"):
    modules = list(model.children())[:-1]
    # print(modules)
    model = nn.Sequential(*modules)
    average_dist_iid = [[], []]
    model.eval()

    with torch.no_grad():
        for batch_idx, batch in tqdm(enumerate(dataloader)):
            batch_img, batch_labels, _ = batch

            batch_img = batch_img.to(device)
            batch_labels = batch_labels.to(device)

            output = model(batch_img)
            output = output.reshape(batch_img.shape[0], -1)
             #if batch_idx == 0:
                # print(output.shape)

            for idx_img_1 in range(batch_img.shape[0] - 1):
                for idx_img_2 in range(idx_img_1 + 1, batch_img.shape[0]):

                    if batch_labels[idx_img_1] == batch_labels[idx_img_2]:
                        if metric == "l2":
                            dist = torch.nn.functional.pairwise_distance(output[idx_img_1], output[idx_img_2])
                        elif metric == "cos":
                            cos = nn.CosineSimilarity(dim=1, eps=1e-6)
                            dist = cos(output[idx_img_1].reshape(1, -1), output[idx_img_2].reshape(1, -1))
                        elif metric == "mink":
                            dist = torch.cdist(output[idx_img_1].reshape(1, -1), output[idx_img_2].reshape(1, -1), p=3)
                        # dist = (output[idx_img_1] - output[idx_img_2]).pow(2).sum(0).sqrt()
                        average_dist_iid[batch_labels[idx_img_1]].append(dist.item())

    return average_dist_iid

def calc_dist(model, dataloader, x, y, metric="l2"): # x-> image to be decided if iid or ood, y-> label
    modules = list(model.children())[:-1]
    model = nn.Sequential(*modules)
    average_dist = 0
    count_labels = 0
    model.eval()
    num_batches = len(dataloader)
    # x, y, _ = img
    print(y)
    x = x.to(device)
    y = y.to(device)
    with torch.no_grad():
        x_shape = list(x.shape)
        x = x.reshape(1, x_shape[0], x_shape[1], x_shape[2])

        output_x = model(x)
        output_x = output_x.reshape(1, -1)
        for batch_idx, batch in tqdm(enumerate(dataloader)):
            batch_img, batch_labels, _ = batch

            batch_img = batch_img.to(device)
            batch_labels = batch_labels.to(device)

            output = model(batch_img)
            output = output.reshape(batch_img.shape[0], -1)
            # if batch_idx == 0:
                # print(output.shape)

            for idx_img_1 in range(batch_img.shape[0]):

                if batch_labels[idx_img_1] == y:

                    if metric == "l2":
                        dist = torch.nn.functional.pairwise_distance(output[idx_img_1], output_x)
                    elif metric == "cos":
                        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
                        dist = cos(output[idx_img_1].reshape(1, -1), output_x.reshape(1, -1))
                    elif metric == "mink":
                        dist = torch.cdist(output[idx_img_1].reshape(1, -1), output_x.reshape(1, -1), p=3)
                    average_dist += dist.item()
                    count_labels += 1

    return average_dist / count_labels


In [32]:
import numpy as np
def calc_mean_std(arr):
    arr = np.asarray(arr)
    return arr.mean(), arr.std()


In [41]:
dist_iid = calc_dist_iid(resnet18_pretrained, id_val_loader)
print(calc_mean_std(dist_iid[0]))
print(calc_mean_std(dist_iid[1]))

347it [00:36,  9.55it/s]

(7.966576247082313, 4.114453328341953)
(49.825562248387, 43.42303952766147)


In [42]:
dist_iid_cos = calc_dist_iid(resnet18_pretrained, id_val_loader, "cos")
print(calc_mean_std(dist_iid_cos[0]))
print(calc_mean_std(dist_iid_cos[1]))

347it [00:51,  6.73it/s]

(0.7919799049652289, 0.1680152192808706)
(0.7590619617230417, 0.17710607193633654)


In [29]:
dist_l2_1_ood=[]
batch_data, batch_labels, _ = next(iter(val_loader))
for idx in range(len(batch_labels)):
    dist_l2_1_ood.append(calc_dist(resnet18_pretrained, id_val_loader, batch_data[idx], batch_labels[idx]))
print(dist_l2_1_ood)

tensor(1)


347it [02:26,  2.36it/s]


tensor(1)


347it [00:19, 17.44it/s]


tensor(1)


...

347it [00:20, 17.24it/s]


tensor(1)


347it [00:20, 17.09it/s]

[48.12580185434881, 47.122555061491404, 48.19732158671518, 48.86843590476756, 48.42316879562676, 47.934727400982375, 48.40140213387857, 48.42589491071027, 48.27477378906638, 48.52999814776862, 48.29174712005708, 47.49868544503276, 48.155919726056545, 47.84677312140183, 47.84191508635555, 47.56111748240753, 48.35410220488408, 48.08161313120373, 48.19475751959513, 48.26106034572395, 48.26563705590794, 47.97635042360272, 48.096771982324725, 48.392889665507454, 48.68382668782692, 48.70096763753377, 48.47466665370229, 48.63761319091284, 46.8111308097578, 48.53838990197721, 48.394299754054416, 48.298474198522065]


NameError: name 'calc_mean_std' is not defined

In [33]:
print(calc_mean_std(dist_l2_1_ood))


(48.176962147803266, 0.435619998337812)


In [34]:

for batch_idx, batch in tqdm(enumerate(val_loader)):
    if batch_idx > 200:
        batch_data, batch_labels, _ = batch
        break


201it [01:41,  1.99it/s]


In [35]:
dist_l2_0_ood=[]
for idx in range(len(batch_labels)):
    dist_l2_0_ood.append(calc_dist(resnet18_pretrained, id_val_loader, batch_data[idx], batch_labels[idx]))
print(dist_l2_0_ood)
print(calc_mean_std(dist_l2_0_ood))

tensor(0)


347it [01:38,  3.53it/s]


tensor(0)


347it [00:19, 17.87it/s]


tensor(0)


...

tensor(0)


347it [00:20, 17.02it/s]


tensor(0)


347it [00:21, 16.45it/s]

[6.447804433305278, 7.410187580465462, 9.312605382544106, 9.999398875687643, 7.107455103621232, 6.62457378703924, 8.163959386986267, 9.453281946286097, 10.92258991177105, 6.886451105716525, 6.65852417166171, 7.121108471517348, 7.596846452209618, 10.719539246147166, 8.061968111302418, 7.656580279540352, 7.612348923062648, 6.898071432828647, 10.099604326586602, 6.972398030911118, 7.802922748255329, 6.866422055513252, 6.751424783661893, 7.658601330204207, 9.276851113886972, 8.483976501528598, 8.22611745022655, 8.345342162569604, 10.057817286399805, 11.670284460715335, 8.98549822165429, 8.240271701318202]
(8.252838336722643, 1.3907146492780709)


In [36]:
dist_cos_1_ood=[]
batch_data, batch_labels, _ = next(iter(val_loader))
for idx in range(len(batch_labels)):
    dist_cos_1_ood.append(calc_dist(resnet18_pretrained, id_val_loader, batch_data[idx], batch_labels[idx], "cos"))

print(dist_cos_1_ood)
print(calc_mean_std(dist_cos_1_ood))

tensor(1)


347it [00:21, 16.17it/s]


tensor(1)


347it [00:21, 16.16it/s]


tensor(1)


...

tensor(1)


347it [00:22, 15.47it/s]


tensor(1)


347it [00:21, 16.02it/s]

[0.175232209817253, 0.4630427185848298, 0.170388258306152, 0.10917338993182181, 0.12266235429505674, 0.22345834297703016, 0.12188075855748703, 0.11155844406321272, 0.12773705047000353, 0.12533079190072502, 0.13358671296433516, 0.3935696503296209, 0.1600833570677566, 0.2524557155848813, 0.2446404269971491, 0.32720591324721354, 0.14727172043930725, 0.1804494578660177, 0.16489989133869928, 0.14942211344689757, 0.12534383641582791, 0.21077857022635704, 0.1809078685274981, 0.12980015278059942, 0.11318587095652129, 0.12595113761969443, 0.10525240928306291, 0.1376744835699961, 0.5055735935551493, 0.11792662925456687, 0.14499225003398764, 0.1414911873307145]
(0.18571647711685704, 0.09975466088155133)


In [37]:
for batch_idx, batch in tqdm(enumerate(val_loader)):
    if batch_idx > 200:
        batch_data, batch_labels, _ = batch
        break



201it [00:09, 22.10it/s]


In [38]:
dist_cos_0_ood=[]
for idx in range(len(batch_labels)):
    dist_cos_0_ood.append(calc_dist(resnet18_pretrained, id_val_loader, batch_data[idx], batch_labels[idx], "cos"))
print(dist_cos_0_ood)
print(calc_mean_std(dist_cos_0_ood))

tensor(0)


347it [00:21, 16.37it/s]


tensor(0)


347it [00:20, 16.68it/s]


tensor(0)


...

tensor(0)


347it [00:22, 15.70it/s]


tensor(0)


347it [00:21, 15.86it/s]

[0.8398456293836027, 0.7464824036629562, 0.40257430049999404, 0.1786766498376539, 0.78359088537249, 0.8219335027535635, 0.8216371987043604, 0.36232815443091587, 0.27823858871997487, 0.8073127665302607, 0.8205473658002671, 0.7756444591836341, 0.7582207652783405, 0.8251768045367371, 0.7733613531406462, 0.7510258162328332, 0.7300574589508723, 0.7989621979373307, 0.18834901913755794, 0.8041467814542013, 0.7413368845991564, 0.8020367983521502, 0.8432519218317668, 0.8200783369959136, 0.4127714201589858, 0.6728487402321728, 0.6402545589860562, 0.6403657719226915, 0.263097822405491, 0.7834267529642515, 0.4978268022985809, 0.6911456807696396]
(0.6586422997832828, 0.2072243901638426)


In [54]:
dist_iid_mink = calc_dist_iid(resnet18_pretrained, id_val_loader, "mink")
print(calc_mean_std(dist_iid_mink[0]))
print(calc_mean_std(dist_iid_mink[1]))

"""
(7.966576247082313, 4.114453328341953)
(49.825562248387, 43.42303952766147)
"""


347it [00:33, 10.28it/s]

(4.242545318183882, 2.0646314975394597)
(19.471529235606923, 15.98058256400135)


'\n(7.966576247082313, 4.114453328341953)\n(49.825562248387, 43.42303952766147)\n'

In [55]:
"""Distanta minkowski"""
dist_mink_1_ood=[]
batch_data, batch_labels, _ = next(iter(val_loader))
for idx in range(len(batch_labels)):
    dist_mink_1_ood.append(calc_dist(resnet18_pretrained, id_val_loader, batch_data[idx], batch_labels[idx], "mink"))

print(dist_mink_1_ood)
print(calc_mean_std(dist_mink_1_ood))


tensor(1)


347it [00:20, 16.75it/s]


tensor(1)


347it [00:19, 17.55it/s]


tensor(1)


...

tensor(1)


347it [00:20, 16.72it/s]


tensor(1)


347it [00:20, 16.69it/s]

[18.806861381425776, 18.428017747809502, 18.84163966417182, 19.13327356422054, 18.931674773571544, 18.72822936117464, 18.924689274611385, 18.92445374192953, 18.864682724112985, 18.975605630730875, 18.870566339177405, 18.56826925163358, 18.818919786833202, 18.70165938285986, 18.697113210815584, 18.586759681133604, 18.904315509091298, 18.789364965038676, 18.834630281620697, 18.86168492073423, 18.860953370570353, 18.750304582305002, 18.795131448590297, 18.919097869308647, 19.04862352648623, 19.062619922998813, 18.942149502691308, 19.042837906859592, 18.314613651035692, 18.980897942213172, 18.91981276443404, 18.881195099049286]
(18.834707774351223, 0.17335022120509108)


In [56]:
for batch_idx, batch in tqdm(enumerate(val_loader)):
    if batch_idx > 200:
        batch_data, batch_labels, _ = batch
        break

201it [00:13, 15.29it/s]


In [57]:
dist_mink_0_ood=[]
for idx in range(len(batch_labels)):
    dist_mink_0_ood.append(calc_dist(resnet18_pretrained, id_val_loader, batch_data[idx], batch_labels[idx], "mink"))
print(dist_mink_0_ood)
print(calc_mean_std(dist_mink_0_ood))

tensor(0)


347it [00:19, 17.46it/s]


tensor(0)


347it [00:20, 16.99it/s]


tensor(0)


...

tensor(0)


347it [00:20, 16.65it/s]


tensor(0)


347it [00:20, 16.63it/s]

[3.4879563477093813, 3.9105290172994667, 4.952353565023015, 5.347901423321754, 3.8520147737457426, 3.6143249717515435, 4.319662275845475, 5.033975590386165, 5.222534278085171, 3.6938475488190647, 3.5639323552180002, 3.878303143508687, 4.128077111906854, 5.6040494969086065, 4.36397144165434, 4.172438359907464, 4.013442903001663, 3.7287927980573636, 5.346719126984802, 3.7637027302136468, 4.20871607063924, 3.765812632485144, 3.662454636745562, 4.0730222112065935, 4.972207514846465, 4.608387525147176, 4.348424479455788, 4.445709322093427, 5.1629670382354655, 6.187736644867445, 4.772453488845223, 4.4486147095607205]
(4.395469860421139, 0.6756717172554383)


In [17]:
def get_class_mean_embeddings(model, train_loader):
    mean_embeds = np.zeros(shape=(2, 512)) # 2 classes, 512 embedding dim
    class_numbers = np.zeros(shape=(2, 1))
    modules = list(model.children())[:-1]
    model = nn.Sequential(*modules)
    model.eval()
    with torch.no_grad():
        for data in tqdm(train_loader):
            inputs, labels, _ = data
            inputs = inputs.to(device)
            labels = labels.numpy()

            # Extract features
            features = torch.flatten(model(inputs), start_dim=1)
            features = features.cpu().numpy()

            for idx in range(len(labels)):
                mean_embeds[labels[idx]] += features[idx]
                class_numbers[labels[idx]] += 1

    mean_embeds /= class_numbers

    return mean_embeds


mean_embeds = get_class_mean_embeddings(resnet18_pretrained, val_loader)
print(mean_embeds)

100%|██████████| 360/360 [03:09<00:00,  1.90it/s]

[[2.50774676 0.10202442 0.39871855 ... 1.77666103 0.95930418 0.37828396]
 [3.07073746 0.07881023 1.8179316  ... 2.53730866 1.39863392 0.2981597 ]]
